## NetCDF data

In [1]:
# %matplotlib inline
# Import the tools we are going to need today:
import os
import datetime
import copy

import numpy as np  # numerical library
import pandas as pd
import xarray as xr  # netCDF library
from scipy import signal
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import netCDF4
import matplotlib.pyplot as plt  # plotting library
import matplotlib.dates as mdates
from matplotlib.colors import BoundaryNorm, LogNorm
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from matplotlib.patches import Rectangle, FancyBboxPatch

import xrft
# Some defaults:
# np.set_printoptions(threshold=50)  # avoid to print very large arrays on screen

from eratools.eratools import era_ds_processor, era_filter, era_cmaps
import LidarT_visualization

plt.style.use('latex_default.mplstyle')

In [2]:
# --- Download of ERA5 data --- #
ml_coeff = pd.read_csv('data/era5-ml-coeff.csv')

era_case = '2014-07-16'
#era_case = '2020-08-07'
#era_case = '2018-06-22'

#ds      = xr.open_dataset('data/ERA5-' + era_case + '-ml.nc')
#ds_T21  = xr.open_dataset('data/ERA5-' + era_case + '-ml-T21.nc')
ds_pv   = xr.open_dataset('data/ERA5-' + era_case + '-pl.nc')
ds_2pvu = xr.open_dataset('data/ERA5-' + era_case + '-pvu.nc')

#ds_interp_path = 'data/ERA5-2014-07-16-ml-interp-T42.nc'
ds_interp_path = 'data/ERA5-' + era_case + '-ml-interp.nc'
use_interpolated_ds = True

output_folder = 'output/' + era_case + '-horizontal/'

if era_case == '2014-07-16':
    lat = -53.75
    lon = 140
    lat_avg = [-52.5,-57.5] # -53.7
    lon_avg = [147.5,152.5] # [290,295] # -67.8 # !!!use eastern coordinates!!!
    western_coordinates = False
    plot_coral_location = False
    lon_range = [80,170] # [80,170]
    lon_eastern = lon
    lidar_label = 'lidar'
else:
    lat = -53.75
    lon = -67.75
    lat_avg = [-52.5,-57.5] # -53.7
    lon_avg = [290,295] # -67.8 # !!!use eastern coordinates!!!
    western_coordinates = True
    plot_coral_location = True     
    lon_range = [-120,-30]
    lon_eastern = 360+lon
    lidar_label = 'CORAL'

lat_range = [-77.5,-32.5]

# lat_range = [-52.5,-57.5] # -53.7
# lon_range_pv = [147.5,152.5]  # [290,295] # -67.8
# lon_range = lon_range_pv
# lon_range = np.array(lon_range_pv) + 360 # -67.8

#### - Preprocessing of ERA5 data - ####
if use_interpolated_ds==False:
    ds = era_ds_processor.prepare_interpolated_ml_ds(ds,ds_T21,ml_coeff,ds_interp_path)
else:
    ds = xr.open_dataset(ds_interp_path)
ds

<xarray.Dataset>
Dimensions:    (time: 72, level: 176, latitude: 241, longitude: 401)
Coordinates:
  * time       (time) datetime64[ns] 2014-07-16 ... 2014-07-18T23:00:00
  * level      (level) float64 0.0 400.0 800.0 ... 6.92e+04 6.96e+04 7e+04
  * latitude   (latitude) float32 -25.0 -25.25 -25.5 ... -84.5 -84.75 -85.0
  * longitude  (longitude) float32 80.0 80.25 80.5 80.75 ... 179.5 179.8 180.0
Data variables:
    t          (time, level, latitude, longitude) float32 ...
    p          (time, level, latitude, longitude) float32 ...
    u          (time, level, latitude, longitude) float32 ...
    v          (time, level, latitude, longitude) float32 ...
    tprime     (time, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-05-24 17:09:10 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [3]:
#### - Further processing of ERA5 data - ####
ds,ds_pv,ds_2pvu = era_ds_processor.processing_data_for_jetexit_comp(ds,ds_pv,ds_2pvu,western_coordinates)

In [4]:
%%capture
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
g = 9.80665
plot_rectangles = False

def major_formatter_lon(x, pos):
    if western_coordinates:
        return "%.f°W" % abs(x)
    else:
        return "%.f°E" % abs(x)

def major_formatter_lat(x, pos):
    return "%.f°S" % abs(x)

###### - Lat-z cross section of tropo and stratosphere - ######
def era5_horizontal_cross_section_composit(ds,t,folder,lat,lon):
    # - FIGURE - #
    projection = ccrs.PlateCarree()
    gskw = {'hspace':0.03, 'wspace':0.03, 'height_ratios': [1,1,1,1.3], 'width_ratios': [7,7,1]} #  , 'width_ratios': [5,5]}
    # fig, axes = plt.subplots(4,3, figsize=(8,9.7), sharex=True, sharey=True, gridspec_kw=gskw, subplot_kw={'projection': ccrs.PlateCarree()}) # original with 80 lon
    fig, axes = plt.subplots(4,3, figsize=(8,8.5), sharex=True, sharey=True, gridspec_kw=gskw, subplot_kw={'projection': ccrs.PlateCarree()}) # 
    # fig.subplots_adjust(bottom=0.05, top=0.95, left=0.05, right=0.95,
    #                 wspace=0.02, hspace=0.02)
    axes[0,2].axis('off')
    axes[1,2].axis('off')
    axes[2,2].axis('off')
    axes[3,2].axis('off')

    gs_top = axes[0,0].get_gridspec()
    gs = axes[3,0].get_gridspec()

    # remove the underlying axes
    for ax in axes[3,0:2]:
        ax.remove()
    axb1 = fig.add_subplot(gs[-1,0])
    axb2 = fig.add_subplot(gs[-1,1])

    # - Define levels and colormaps to plot for relevant variables - #
    # levels = [40000,30000,20000,12000]
    alt_range = [3,31] # [3,27]
    lat2 = -60
    # levels = [24000,18000,12000], [30000,22000,14000]
    levels = [26000,20000,14000]
    h_str = ['z: {}km'.format(int(levels[0]/1000)),'z: {}km'.format(int(levels[1]/1000)),'z: {}km'.format(int(levels[2]/1000))]
    pvlev = [-4,-3,-2,-1]

    # - Define levels and colormaps to plot for relevant variables - #
    CLEV_11 = [-11,-9,-7,-5,-3,-1,1,3,5,7,9,11]
    CLEV_11_LABELS = [-9,-7,-5,-3,-1,1,3,5,7,9]
    # CLEV_11_LABELS = [-9,-5,-1,1,5,9]
    # CLEV_32 = [-32,-16,-8,-4,-2,-1,-0.5,-0.25,0.25,0.5,1,2,4,8,16,32]
    # CLEV_32_LABELS = [-16,-4,-1,-0.25,0.25,1,4,16]
    CLEV_32   = [-32,-16,-8,-4,-2,-1,-0.5,0.5,1,2,4,8,16,32] # 32
    CLEV_32   = [-32,-16,-8,-4,-2,-1,1,2,4,8,16,32] # 32
    CLEV_32_LABELS = [-16,-4,-1,1,4,16]
    CLEV_64 = [-64,-32,-16,-8,-4,-2,-1,-0.5,0.5,1,2,4,8,16,32,64]
    CLEV_64_LABELS = [-32,-8,-2,-0.5,0.5,2,8,32]
    clev_lin = [-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8]
    clev_lin = [-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8]
    clev_lin = [-5,-4.5,-4,-3.5,-3,-2.5,-2,-1.5,-1,1,1.5,2,2.5,3,3.5,4,4.5,5]
    blue = 'mediumblue'
    red  = 'firebrick'
    clev_colors = [blue,blue,blue,blue,blue,blue,blue,blue,blue,red,red,red,red,red,red,red,red,red]
    # clev = CLEV_11
    clev = clev_lin
    clev_l = CLEV_32_LABELS
    cmap_st = plt.get_cmap('RdBu_r')
    norm_st = BoundaryNorm(boundaries=clev, ncolors=cmap_st.N, clip=True)
    
    thlev = np.arange(220,600,5)
    thlev_labels = np.arange(220,600,40)
    # thlev_st = np.exp(5+0.03*np.arange(1,120,4))
    thlev_st = np.exp(5+0.03*np.arange(1,120,1))
    thlev_st_labels = np.exp(5+0.03*np.arange(1,120,3))

    ulev = np.arange(30,180,10)

    # pressure_levels = np.arange(0,1000,1) * 100
    # pressure_levels = np.exp(np.arange(-2,50,0.05))
    pressure_levels = np.logspace(-1,5,500)

    geop_levels = np.arange(7500,15000,70) # maybe choose small range here
    
    wind_levels = np.arange(35,70,5)
    cmap = plt.get_cmap('Greens') # Greys, Greens
    norm = BoundaryNorm(boundaries=wind_levels , ncolors=cmap.N, clip=True)
    wind_levels_vert = np.arange(30,110,10)
    cmap_vert = plt.get_cmap('Greys') # Greys
    norm_vert = BoundaryNorm(boundaries=wind_levels_vert, ncolors=cmap_vert.N, clip=True)
    nbarbs = 25
    met_level = 300
    lw_cut  = 2
    lw_tprime = 0.6
    # h_fmt = mdates.DateFormatter('%m-%d %H:%M')
    
    for j in range(0,3):
        ax_tpj = axes[j,0]
        ax_pvu = axes[j,1]
        tmp_level = levels[j]

        #### - TPJ Jet - ####
        cont_z   = ax_tpj.contour(ds_pv.longitude_plot, ds_pv.latitude, ds_pv['z'].sel(level=met_level)[t,:,:]/g, colors='dimgray', levels=geop_levels, linewidths=0.5)
        contf_wind  = ax_tpj.contourf(ds_pv.longitude_plot, ds_pv.latitude, ds_pv['u_horiz'].sel(level=met_level)[t,:,:], cmap=cmap, norm=norm, levels=wind_levels,extend='both')

        # Replace everything below threshold with NAN -> transparent??
        cont_tprime  = ax_tpj.contour(ds.longitude_plot,  ds.latitude, ds['tprime'][t,:,:,:].sel(level=tmp_level), colors=clev_colors, levels=clev, linewidths=lw_tprime, extend='both')
        # contf_tprime = ax_tpj.contourf(ds.longitude,  ds.latitude, ds['tprime_m'][t,:,:,:].sel(level=tmp_level), cmap=cmap_st, norm=norm_st, levels=clev, extend='both', alpha=1)


        # barbs_met  = ax_tpj.barbs(ds_pv.longitude[::nbarbs], ds_pv.latitude[::nbarbs], ds_pv['u'].sel(level=met_level)[t,::nbarbs,::nbarbs], ds_pv['v'].sel(level=met_level)[t,::nbarbs,::nbarbs], transform=projection, length=5, linewidth=0.6)
        # ax_pvu.clabel(cont_met, fmt= '%1.0fm', inline=True, fontsize=9)

        ax_tpj.axhline(lat, color='black', ls='--', lw=lw_cut)
        if plot_coral_location:
            ax_tpj.axvline(lon, color='black', ls='--', lw=lw_cut)

        ax_tpj.coastlines(lw=0.5)
        gls = ax_tpj.gridlines(draw_labels=True, x_inline=False, y_inline=False)
        gls.right_labels=False
        gls.top_labels=False
        if j!=3:
            gls.bottom_labels=False

        #### - 2PVU - ####
        z_levs = [3,4,5,6,7,8,9,10,11,12]
        contf_pvu    = ax_pvu.contourf(ds_2pvu.longitude_plot, ds_2pvu['latitude'], ds_2pvu['z'][t,:,:]/(1000*g), levels=z_levs, transform=projection,cmap='turbo', extend='both') # Spectral_r
        cont_tprime  = ax_pvu.contour(ds.longitude_plot,  ds.latitude, ds['tprime'][t,:,:,:].sel(level=tmp_level), colors='k', levels=clev_lin, linewidths=lw_tprime-0.1, extend='both')

        #contf_wind_strat = ax_pvu.contourf(ds.longitude_plot, ds.latitude, ds['u_horiz'].sel(level=tmp_level)[t,:,:], cmap=cmap_vert, norm=norm_vert, levels=wind_levels_vert,extend='both')
        cont_p      = ax_pvu.contour(ds.longitude_plot, ds.latitude, ds['p'].sel(level=tmp_level)[t,:,:],colors='dimgray', levels=pressure_levels, linewidths=0.5)
        #cont_tprime = ax_pvu.contour(ds.longitude_plot,  ds.latitude, ds['tprime'][t,:,:,:].sel(level=tmp_level), colors=clev_colors, levels=clev, linewidths=lw_tprime, extend='both')

        ax_pvu.axhline(lat2, color='black', ls='--', lw=lw_cut)

        ax_pvu.coastlines(lw=0.5)
        gls = ax_pvu.gridlines(draw_labels=True, x_inline=False, y_inline=False)
        gls.left_labels=False
        gls.right_labels=False
        gls.top_labels=False
        if j!=3:
            gls.bottom_labels=False
    
        # - Numbering - #
        # - Labels - #
        numb_str = ['a','b','c','d','e','f','g','h']
        ypp = 0.89
        xpp = 0.033
        ax_tpj.text(xpp, ypp, numb_str[2*j], transform=ax_tpj.transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
        ax_pvu.text(xpp, ypp, numb_str[2*j+1], transform=ax_pvu.transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
        ax_tpj.text(xpp, 0.07, h_str[j], transform=ax_tpj.transAxes, weight='bold', bbox={"boxstyle" : "round", "lw":0.67, "facecolor":"white", "edgecolor":"black"})

    # - Draw rectangle around GWs above fold - #
    if plot_rectangles:
        # rect0 = Rectangle([126,-61],15,13,linewidth=2.5, linestyle=(0, (1, 1)), edgecolor='black', facecolor='none')
        rect0 = FancyBboxPatch([126,-61],15,13, linewidth=2.5, linestyle=(0, (1, 1)), edgecolor='black', boxstyle='round', facecolor='none')
        rect3 = FancyBboxPatch([126,-61],15,13, linewidth=2.5, linestyle=(0, (1, 1)), edgecolor='black', boxstyle='round', facecolor='none')
        rect1 = copy.copy(rect0)
        rect2 = copy.copy(rect0)
        axes[2,0].add_patch(rect0)
        axes[1,0].add_patch(rect1)
        axes[0,0].add_patch(rect2)
        axes[2,1].add_patch(rect3)

    # ax_tpj.set_xlim([ds_2pvu.longitude[0],ds_2pvu.longitude[-1]])
    ax_tpj.set_xlim(lon_range)
    ax_tpj.set_ylim(lat_range)
    # ax_pvu.set_xlim([ds_2pvu.longitude[0],ds_2pvu.longitude[-1]])
    # ax_pvu.set_ylim([-77.5,-32.5])
    fig.subplots_adjust(bottom=0, top=0.95, left=0, right=1,
                    wspace=0.04, hspace=0.04)
    

    ### - Vertical Cross Sections - ###
    for ii in range(0,2):
        if ii==0: 
            axb = axb1
            lat_temp = lat
        else:
            axb = axb2
            lat_temp = lat2

        ## Tprime
        # nx_avg = 50 # 50 -> approx. lambdax=750km assuming 1°=60km, 60->900km
        # tprime_lon_z, tprime_lat_z = era_filter.horizontal_temp_filter(ds,t,lat,lon, nx_avg=nx_avg)
        tprime_lon_z_T21 = ds['tprime'][t,:,:,:].sel(latitude=lat_temp,method='nearest').values

        # contf_tprime = axb1.contourf(ds['longitude'], ds['level']/1000, tprime_lon_z_T21, levels=clev, cmap=cmap_st, norm=norm_st, extend='both')
        cont_tprime  = axb.contour(ds.longitude_plot, ds['level']/1000, tprime_lon_z_T21, colors=clev_colors, levels=clev, linewidths=lw_tprime+0.1, extend='both') # lw=0.8

        ## Theta
        cont_th0  = axb.contour(ds.longitude_plot, ds['level']/1000, ds['th'].sel(latitude=lat_temp)[t,:,:], colors='dimgray', levels=thlev_st, linewidths=0.3)
        # axb1.clabel(cont_th0, thlev_labels, fmt= '%1.0fK', inline=True, fontsize=9, manual=th_label_lon)

        ## PVU lines
        contb0 = axb.contour(ds_pv['longitude_3d'][t,:,0,:], ds_pv['z'].sel(latitude=lat_temp)[t,:,:] / (g*1000), ds_pv['pv'].sel(latitude=lat)[t,:,:] * 10**(6), colors=['k', 'k', 'limegreen', 'k'], linestyles='solid', linewidths=[1.5, 1.5, 2.5, 1.5], levels=pvlev)
        axb.clabel(contb0, [-4,-3,-2,-1], fmt= '%1.0f', inline=True)

        ## Wind
        # cont_v  = axb.contour(ds['longitude'], ds['level']/1000, ds['u_horiz'].sel(latitude=lat)[t,:,:], colors='k', levels=ulev, linewidths=0.9, linestyles='--')
        # axb.clabel(cont_v, ulev, fmt= '%1.0f', inline=True, fontsize=9)
        contf_wind_vert  = axb.contourf(ds.longitude_plot, ds['level']/1000, ds['u_horiz'].sel(latitude=lat_temp)[t,:,:], cmap=cmap_vert, norm=norm_vert, levels=wind_levels_vert, alpha=0.95, extend='both')
        # contf_wind_vert  = axb.contourf(ds.longitude_plot, ds['level']/1000, ds['u_horiz'].sel(latitude=lat_temp)[t,:,:], cmap=cmap, norm=norm, levels=wind_levels, extend='both')


        axb.axhline(levels[0]/1000, color='black', ls='--', lw=lw_cut)
        axb.axhline(levels[1]/1000, color='black', ls='--', lw=lw_cut)
        axb.axhline(levels[2]/1000, color='black', ls='--', lw=lw_cut)

        axb.set_ylim(alt_range)
        axb.set_xlim(lon_range)

        axb.yaxis.set_minor_locator(AutoMinorLocator()) 
        axb.xaxis.set_minor_locator(AutoMinorLocator())

        axb.set_xlabel('longitude / °')

        axb.xaxis.set_major_formatter(major_formatter_lon)
        axb.xaxis.set_label_position('bottom') 
        axb.tick_params(which='both', top=True, labelbottom=True,labeltop=False)

        th_y_pos = np.linspace(13,30,6)
        th_label_lon = []
        for lab in th_y_pos:
            th_label_lon.append((lon-40,lab))
        axb.clabel(cont_th0, thlev_st_labels, fmt= '%1.0fK', inline=True, fontsize=9, manual=th_label_lon)


    # - Draw rectangle around GWs above fold - #
    if plot_rectangles:
        rect0 = FancyBboxPatch([126,12],15,17.5,linewidth=2.5, linestyle=(0, (1, 1)), edgecolor='black', boxstyle='round', facecolor='none')
        axb1.add_patch(rect0)

    axb1.text(0.033, 0.05, 'y: 53.75°S', transform=axb1.transAxes, weight='bold', bbox={"boxstyle" : "round", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    axb2.text(0.033, 0.05, 'y: 60°S', transform=axb2.transAxes, weight='bold', bbox={"boxstyle" : "round", "lw":0.67, "facecolor":"white", "edgecolor":"black"})


    if plot_coral_location:
        axb1.axvline(lon, color='black', ls='--', lw=lw_cut)
    axb1.set_ylabel('altitude / km')
    # axb1.tick_params(which='both', labelbottom=False,labeltop=False)
    axb2.tick_params(which='both',labeltop=False,labelleft=False)

    j=3
    ypp=0.915
    xpp=0.033
    axb1.text(xpp, ypp, numb_str[2*j], transform=axb1.transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    axb2.text(xpp, ypp, numb_str[2*j+1], transform=axb2.transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    
    # timestamp_str = str(ds.time[t].values)[0:19].replace('T',' ')
    # ax_lid.text(0.4, 1.13, timestamp_str, transform=ax_lid.transAxes)
    

    # - COLORBARS - #
    # cbar = fig.colorbar(contf_st, ax=axes[0:2,2], location='right', ticks=clev_l, shrink=0.6, fraction=1, aspect=25)
    # cbar.set_label(r"$T'$ / K")

    # - 2PVU - #
    cbar = fig.colorbar(contf_pvu, ax=axes[0:2,2], location='right', fraction=1, shrink=0.75, aspect=27)
    cbar.set_label('height of the dynamical tropopause / km')
    cb_position = cbar.ax.get_position()
    cbar.ax.set_position([cb_position.x0, cb_position.y0+0.04, cb_position.width, cb_position.height])

    # - 300hPa wind - #
    cbar = fig.colorbar(contf_wind, ax=axes[1:3,2], location='right', fraction=1, shrink=0.65, aspect=23)
    cbar.set_label('horizontal wind at 300hPa / ms$^{-1}$')
    cb_position = cbar.ax.get_position()
    cbar.ax.set_position([cb_position.x0+0.003, cb_position.y0-0.065, cb_position.width, cb_position.height])

    # - Absolute wind - #
    cbar = fig.colorbar(contf_wind_vert, ax=axes[3,2], location='right', fraction=1, shrink=0.95, aspect=23)
    cbar.set_label('horizontal wind / ms$^{-1}$')
    
    timestamp_str = datetime.datetime.strftime(pd.to_datetime(str(ds['time'][t].values)), format='%Y-%m-%d %H:%M')
    axes[0,0].text(0.975, 0.945, timestamp_str, transform=axes[0,0].transAxes, verticalalignment='top', horizontalalignment='right', bbox={"boxstyle" : "round", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    # fig.suptitle(datetime_str)

    fig_name = folder + 'era5_horiz_ana' + '{:02d}'.format(t) + '.png'
    fig.savefig(fig_name, facecolor='w', edgecolor='w', format='png', dpi=200, bbox_inches='tight') # orientation='portrait'

##### - RUN - #####
t  = 33 # 2018
#t = 24 # 55 # 2020
#t = 41 # 2014
era5_horizontal_cross_section_composit(ds,t,output_folder,lat,lon)

for tstep in range(np.shape(ds['t'])[0]):
    era5_horizontal_cross_section_composit(ds,tstep,output_folder,lat,lon)

In [6]:
import imageio

image_folder=output_folder
fps=6

# Get a list of file names in the folder
file_names = sorted(os.listdir(image_folder))

# Read and append each image to images list
images = []
for file_name in file_names:
    if file_name.endswith(".png"):  # Adjust the file extension if needed
        image_path = os.path.join(image_folder, file_name)
        image = imageio.imread(image_path)
        images.append(image)

# Create the GIF using imageio
imageio.mimsave(image_folder + "/era5_sequence.gif", images, duration=1/fps, palettesize=256/2)  # loop=0, quantizer="nq", palettesize=256
print("GIF created successfully!")

/var/folders/5n/r616r2rx3710m3kyf90_xwdr0000gn/T/ipykernel_58916/269154493.py:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


GIF created successfully!
